<a href="https://colab.research.google.com/github/seungeunlee00/JUNIA/blob/main/MH/Tabu_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabu Search Algorithm(TS)

In [7]:
from google.colab import drive # 드라이브 마운트 필수
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# load data from tsp file
import numpy as np

tsp_data = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/JUNIA/Metaheuristics/gr17.2085.tsp')
print(tsp_data)

[[  0. 633. 257.  91. 412. 150.  80. 134. 259. 505. 353. 324.  70. 211.
  268. 246. 121.]
 [633.   0. 390. 661. 227. 488. 572. 530. 555. 289. 282. 638. 567. 466.
  420. 745. 518.]
 [257. 390.   0. 228. 169. 112. 196. 154. 372. 262. 110. 437. 191.  74.
   53. 472. 142.]
 [ 91. 661. 228.   0. 383. 120.  77. 105. 175. 476. 324. 240.  27. 182.
  239. 237.  84.]
 [412. 227. 169. 383.   0. 267. 351. 309. 338. 196.  61. 421. 346. 243.
  199. 528. 297.]
 [150. 488. 112. 120. 267.   0.  63.  34. 264. 360. 208. 329.  83. 105.
  123. 364.  35.]
 [ 80. 572. 196.  77. 351.  63.   0.  29. 232. 444. 292. 297.  47. 150.
  207. 332.  29.]
 [134. 530. 154. 105. 309.  34.  29.   0. 249. 402. 250. 314.  68. 108.
  165. 349.  36.]
 [259. 555. 372. 175. 338. 264. 232. 249.   0. 495. 352.  95. 189. 326.
  383. 202. 236.]
 [505. 289. 262. 476. 196. 360. 444. 402. 495.   0. 154. 578. 439. 336.
  240. 685. 390.]
 [353. 282. 110. 324.  61. 208. 292. 250. 352. 154.   0. 435. 287. 184.
  140. 542. 238.]
 [324. 638

In [9]:
import queue

def pickneighbor(s):
    index = np.random.choice(range(17), 2, replace=False) # pick randomly 2 indices in [0,16]
    s[index[0]],s[index[1]] = s[index[1]],s[index[0]] # swap values located at these indices
    return s

def objectivefunction(s):
    cost = 0
    for i in range(s.shape[0]-1):
        cost = cost + tsp_data[s[i]][s[i+1]]
    
    cost = cost + tsp_data[s[-1]][s[0]]
    return cost
    
def tabu_neighbor(s, tabu_list):
    
    for sol in tabu_list.queue:
        if((sol==s).all()):
            return True
        
    return False
    
def best_admessible_neighbor(s,tabu_list):
   
    while True:
        neighbor_sol = pickneighbor(np.copy(s))
        tabu = tabu_neighbor(neighbor_sol, tabu_list)
        
        if (objectivefunction(neighbor_sol)<objectivefunction(s)):
            if tabu:
                return neighbor_sol, True
            else:
                return neighbor_sol, False
        else:
            if not(tabu):
                return neighbor_sol, False

In [10]:
import queue
import matplotlib.pyplot as plt

tabu_list = queue.Queue(5) # FIFO queue of 5 items max
costs_hist = []

# random initial solution and its cost
s = np.random.choice(range(17), 17, replace=False) #cities in [0,16]
cost_s = objectivefunction(s)
# update tabu list
tabu_list.put(np.copy(s))
# save history of costs
costs_hist.append(cost_s)
# consider solution s as the best solution for now
best_s = np.copy(s) # hard copy
best_cost = cost_s
total_it = 0 # max iterations
while total_it < 5000:
    adm_s, tabu = best_admessible_neighbor(np.copy(s),tabu_list)
    cost_adm_s = objectivefunction(adm_s)
    costs_hist.append(cost_adm_s)
    
    # update s
    s = np.copy(adm_s)
    
    if (not(tabu)): # adm_s not satisfying an aspiration criterion
        
        if (best_cost > cost_adm_s): # update best sol if necessary
                best_s = np.copy(adm_s) # update best solution
                best_cost = cost_adm_s
        
        # update tabu list by adding adm_s as a visited solution
        if not(tabu_list.full()): 
            tabu_list.put(np.copy(adm_s))
        else:
            tabu_list.get() # get an item to free a place
            tabu_list.put(np.copy(adm_s))
            
    total_it = total_it+1
        
    
    
print ('best sol, cost {} {}'.format(best_s,best_cost))

best sol, cost [10  8 11  7 14 13  0 15 12  6 16  3  5  2  9  4  1] 3088.0
